In [1]:
import pandas as pd
import numpy as np

In [2]:
# 1. Load the patient dataset and show summary
patient_df = pd.read_csv('Patient_Data.csv')
billing_df = pd.read_csv('Billing_Data.csv')

In [3]:
print("Patient Dataset Info:\n")
patient_df.info()

Patient Dataset Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [4]:
# 2. Select only columns relevant for billing
billing_columns = ['PatientID', 'Department', 'Doctor', 'BillAmount']
billing_relevant = patient_df[billing_columns]

In [5]:
# 3. Drop administrative columns
admin_columns = ['ReceptionistID', 'CheckInTime']
patient_df = patient_df.drop(columns=admin_columns, errors='ignore')

In [6]:
# 4. Total bill amount per department
bill_by_department = (
billing_relevant
.groupby('Department')['BillAmount']
.sum()
.reset_index()
)

In [7]:
# 5. Remove duplicate patient records based on PatientID
patient_df = patient_df.drop_duplicates(subset='PatientID')

In [8]:
# 6. Fill missing BillAmount values with mean
mean_bill = billing_relevant['BillAmount'].mean()
billing_relevant['BillAmount'] = billing_relevant['BillAmount'].fillna(mean_bill)

In [9]:
# 7. Merge billing dataset with patient dataset on PatientID
merged_df = pd.merge(patient_df, billing_df, on='PatientID', how='inner')

In [10]:
# 8. Concatenate new patients (row-wise)
# new_patients_df = pd.read_csv('New_Patients_Week.csv')
# final_patient_df = pd.concat([merged_df, new_patients_df], axis=0, ignore_index=True)
final_patient_df = merged_df.copy()

In [11]:
# 9. Concatenate new billing category columns (column-wise)
new_billing_cols = pd.DataFrame({
'InsuranceCovered': ['Yes'] * len(final_patient_df),
'FinalAmount': final_patient_df['BillAmount'] * 0.9
})

In [12]:
final_df = pd.concat([final_patient_df, new_billing_cols], axis=1)

In [13]:
# Final Output
print("\nFinal Cleaned Dataset Preview:\n")
print(final_df.head())


Final Cleaned Dataset Preview:

   PatientID     Name   Department     Doctor  BillAmount InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith      5000.0             2000   
1        102      Bob    Neurology   Dr. John         NaN             1500   
2        103  Charlie  Orthopedics    Dr. Lee      7500.0             2500   
3        104    David   Cardiology  Dr. Smith      6200.0             3000   
4        105      Eva  Dermatology   Dr. Rose         NaN             1000   

   FinalAmount InsuranceCovered  FinalAmount  
0         3000              Yes       4500.0  
1         3500              Yes          NaN  
2         5000              Yes       6750.0  
3         3200              Yes       5580.0  
4         4000              Yes          NaN  


In [14]:
print("\nDepartment-wise Revenue:\n")
print(bill_by_department)


Department-wise Revenue:

    Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0
